# Multi-Retailer Fashion Assortment & White-Space Analysis

**Datasets:** SSENSE, Net-a-Porter, Mr Porter, Vestiaire Collective  
**Goal:** Build a unified product dataset to analyze assortment (categories) and price bands across multiple fashion retailers and identify white-spaces.

**Key Steps**
1. Load and standardize raw datasets from 4 retailers.
2. Create a unified schema:
   - retailer, brand, product_name, description, category_raw, gender, price_usd
3. Engineer:
   - category_main (clean category)
   - price_band (price segments)
4. Build:
   - Assortment matrix (retailer × category_main)
   - White-space matrix (category_main × price_band)


In [14]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 20)


In [ ]:
RAW_PATH = "../data/raw/"

ssense = pd.read_csv(RAW_PATH + "ssense_dataset.csv")
netaporter = pd.read_csv(RAW_PATH + "net-a-porter.csv")
mrporter = pd.read_csv(RAW_PATH + "mr-porter.csv")
vestiaire = pd.read_csv(RAW_PATH + "vestiaire.csv")

for name, df in {
    "SSENSE": ssense,
    "Net-a-Porter": netaporter,
    "Mr Porter": mrporter,
    "Vestiaire": vestiaire,
}.items():
    print(f"=== {name} ===")
    print("Shape:", df.shape)
    print("Columns:", list(df.columns))
    print()


In [ ]:
# Standardize Vestiaire Collective to unified schema

vestiaire_std = vestiaire.copy()

vestiaire_std["retailer"] = "Vestiaire"
vestiaire_std["brand"] = vestiaire_std["brand_name"]
vestiaire_std["description"] = vestiaire_std["product_description"]
vestiaire_std["category_raw"] = vestiaire_std["product_category"]
vestiaire_std["gender"] = vestiaire_std["product_gender_target"]

vestiaire_std = vestiaire_std[
    ["retailer", "brand", "product_name", "description",
     "category_raw", "gender", "price_usd"]
]

vestiaire_std.head()


,retailer,brand,product_name,description,category_raw,gender,price_usd
0,Vestiaire,Miu Miu,Wool mini skirt Miu Miu Grey size S Internatio...,Miu Miu – Pleated mini skirt Size: 36 (S) Wai...,Women Clothing,Women,272.92
1,Vestiaire,Barbara Bui,Jacket Barbara Bui Navy size 42 FR in Cotton,For selling nice women's suit Barbara Bui size...,Women Clothing,Women,127.80
2,Vestiaire,Comme Des Garcons,Wool coat Comme Des Garcons White size S Inter...,Magnificent boiled wool coat. I bought it in t...,Women Clothing,Women,1278.00
3,Vestiaire,MSGM,Mini skirt MSGM Black size 38 IT in Polyester,MSGM Skirt Black Printed Raw-Edge & Embroidere...,Women Clothing,Women,163.90
4,Vestiaire,LVIR,Vegan leather trousers LVIR Black size 36 FR i...,LVIR black grained faux leather trousers size ...,Women Clothing,Women,119.72


In [ ]:
def standardize_simple_retailer(df, retailer_name):
    """
    Standardize a simple retailer dataset (brand, description, price_usd, type)
    to the unified schema.
    """
    df_std = df.copy()
    df_std["retailer"] = retailer_name
    df_std["product_name"] = df_std["description"]      # no separate name column
    df_std["category_raw"] = df_std["type"]             # type -> category_raw
    df_std["gender"] = np.nan                           # not available

    df_std = df_std[
        ["retailer", "brand", "product_name", "description",
         "category_raw", "gender", "price_usd"]
    ]
    return df_std


ssense_std = standardize_simple_retailer(ssense, "SSENSE")
netaporter_std = standardize_simple_retailer(netaporter, "Net-a-Porter")
mrporter_std = standardize_simple_retailer(mrporter, "Mr Porter")

ssense_std.head()


,retailer,brand,product_name,description,category_raw,gender,price_usd
0,SSENSE,Rick Owens,Black Moncler Edition Down Coat,Black Moncler Edition Down Coat,mens,NaN,2585
1,SSENSE,Rick Owens,Black Moncler Edition Cyclopic Down Coat,Black Moncler Edition Cyclopic Down Coat,mens,NaN,2405
2,SSENSE,Rick Owens,Black Moncler Edition Cyclopic Down Jacket,Black Moncler Edition Cyclopic Down Jacket,mens,NaN,3160
3,SSENSE,Rick Owens,Taupe & Green Moncler Edition Down Coat,Taupe & Green Moncler Edition Down Coat,mens,NaN,3505
4,SSENSE,Rick Owens,Taupe & Green Moncler Edition Cyclopic Down Ja...,Taupe & Green Moncler Edition Cyclopic Down Ja...,mens,NaN,2885


In [ ]:
fashion_all = pd.concat(
    [ssense_std, netaporter_std, mrporter_std, vestiaire_std],
    ignore_index=True
)

fashion_all.shape, fashion_all["retailer"].value_counts()


((963941, 7),
 retailer
 Vestiaire       900514
 Net-a-Porter     23161
 Mr Porter        20347
 SSENSE           19919
 Name: count, dtype: int64)

In [ ]:
import os

os.makedirs("../data/processed", exist_ok=True)

fashion_all.to_csv("../data/processed/fashion_all.csv", index=False)


In [ ]:
def map_category_main(row):
    text = (str(row["category_raw"]) + " " + str(row["product_name"])).lower()

    # Outerwear
    if any(k in text for k in ["coat", "parka", "puffer", "trench", "jacket", "blazer"]):
        return "Outerwear"
    # Dresses
    elif "dress" in text:
        return "Dresses"
    # Skirts
    elif "skirt" in text:
        return "Skirts"
    # Pants & Shorts
    elif any(k in text for k in ["jeans", "denim", "trouser", "pants", "pant ", "chino", "legging", "shorts"]):
        return "Pants & Shorts"
    # Knitwear & Sweats
    elif any(k in text for k in ["sweater", "jumper", "cardigan", "knit", "pullover", "hoodie", "sweatshirt"]):
        return "Knitwear & Sweats"
    # Tops & Shirts
    elif any(k in text for k in ["shirt", "top", "blouse", "t-shirt", "tshirt", "polo", "bodysuit", "tank"]):
        return "Tops & Shirts"
    # Shoes
    elif any(k in text for k in ["sneaker", "trainer", "shoe", "boot", "loafer", "heel", "sandal", "pump", "flat", "moccasin", "espadrille"]):
        return "Shoes"
    # Bags
    elif any(k in text for k in ["bag", "backpack", "tote", "clutch", "crossbody", "belt bag", "shoulder bag"]):
        return "Bags"
    # Accessories
    elif any(k in text for k in ["scarf", "hat", "cap", "beanie", "glove", "sock", "sunglass", "belt", "wallet", "tie"]):
        return "Accessories"
    else:
        return "Other"


fashion_all["category_main"] = fashion_all.apply(map_category_main, axis=1)
fashion_all["category_main"].value_counts()


category_main
Shoes                273756
Other                192118
Accessories          179604
Outerwear             98920
Pants & Shorts        67195
Tops & Shirts         60247
Knitwear & Sweats     45341
Dresses               33121
Skirts                 9243
Bags                   4396
Name: count, dtype: int64

In [ ]:
fashion_all["price_usd"] = pd.to_numeric(fashion_all["price_usd"], errors="coerce")

price_bins = [0, 100, 200, 300, 500, 1000, np.inf]
price_labels = ["<100", "100–199", "200–299", "300–499", "500–999", "1000+"]

fashion_all["price_band"] = pd.cut(
    fashion_all["price_usd"],
    bins=price_bins,
    labels=price_labels,
    right=False
)

fashion_all["price_band"].value_counts().sort_index()


price_band
<100       282615
100–199    236835
200–299    134138
300–499    138206
500–999    106743
1000+       65404
Name: count, dtype: int64

In [ ]:
assortment_by_retailer = (
    fashion_all
    .groupby(["retailer", "category_main"])
    .size()
    .reset_index(name="sku_count")
)

assortment_pivot = (
    assortment_by_retailer
    .pivot(index="category_main", columns="retailer", values="sku_count")
    .fillna(0)
    .astype(int)
)

assortment_pivot


retailer,Mr Porter,Net-a-Porter,SSENSE,Vestiaire
category_main,,,,
Accessories,2023,1316,2028,174237
Bags,594,1828,1183,791
Dresses,0,3934,552,28635
Knitwear & Sweats,2867,1770,2140,38564
Other,2515,2631,2658,184314
Outerwear,2732,2183,2764,91241
Pants & Shorts,3473,2810,3043,57869
Shoes,1976,3083,2032,266665
Skirts,4,1012,439,7788
